<a href="https://colab.research.google.com/github/EthanXC/RedditProject/blob/main/Copy_of_Version_0_0_RedditProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import numpy as np
import pandas as pd

In [16]:
!pip install datasets
# FYI the accellerate -u is very important to avoid version issues with torch
!pip install transformers
!pip install accelerate -U

In [17]:
from datasets import load_dataset

In [18]:
dataset = load_dataset("fddemarco/pushshift-reddit-comments", streaming = True)

Resolving data files:   0%|          | 0/50 [00:00<?, ?it/s]

In [19]:
print(dataset)

IterableDatasetDict({
    train: IterableDataset({
        features: ['author', 'body', 'controversiality', 'created_utc', 'link_id', 'score', 'subreddit', 'subreddit_id', 'id'],
        n_shards: 50
    })
})


In [20]:
it = iter(dataset['train'])


In [99]:
extract_data = next(it)
#print(extract_data)

print(extract_data['body'])

and it would be called . . . . .


In [100]:
data = {
    'column1_name': ['text'],
    'column2_name': ['score'],
    'column3_name': ['labels']
}

df = pd.DataFrame(data)

In [101]:
# Initialize an empty DataFrame with columns
df = pd.DataFrame(columns=['body', 'score', 'subreddit'])

# Your code for iterating through the data and adding rows to the DataFrame
for i in range(10000):  # only 10,000 rows
    extract_data = next(it)
    body = extract_data['body']
    score = extract_data['score']
    subreddit = extract_data['subreddit']

    new_row_data = {'body': body, "score": score, "subreddit": subreddit}
    df.loc[len(df)] = new_row_data

    if i % 1000 == 0:
        print(i)
        print(new_row_data)


0
{'body': 'that is nice of aa to do that . on a related note , checkpoints are a bunch of shit . edit : aaa = p', 'score': 1, 'subreddit': 'orangecounty'}
1000
{'body': "also , the space pen thing is [ a myth ] ( http : / / www . snopes . com / business / genius / spacepen . asp ) . they used them , but they didn ' t invent them or pay to develop them .", 'score': 1, 'subreddit': 'askscience'}
2000
{'body': 'wait a minute how would you not know the price ? do i smell a karma whore ? sniff sniff', 'score': 3, 'subreddit': 'glassheads'}
3000
{'body': 'i offered on the indie bundle so i need that guy to respond first . how much did you want for it , out of curiosity ?', 'score': 1, 'subreddit': 'SteamTrade'}
4000
{'body': 'pauly ?', 'score': 2, 'subreddit': 'movies'}
5000
{'body': "looks like [ lucy thai ] ( http : / / www . xvideos . com / video1045083 / double _ creampie _ lucy _ thai ) to me . * video to show similarities only , didn ' t watch enough to see if the scene was in it .", 

In [102]:
df.head()


,body,score,subreddit
0,that is nice of aa to do that . on a related n...,1,orangecounty
1,"the "" forgotten "" war is known by that moniker...",2,AskReddit
2,yet people build cobblestone generators .,1,MinecraftInventions
3,happy new year from england !,10,fffffffuuuuuuuuuuuu
4,[ there is one i might be able to get ] ( http...,1,cars


In [103]:
from transformers import RobertaTokenizer, RobertaModel
import torch
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
# Load DistilBERT model and tokenizer filling out the code below based on the documentation HINT: Click on on use in transformers and the code will be there
#model_name = "Roberta model"# code unnessisary but I like to use it to store the string name to make the code cleaner
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [104]:
nan_rows = df[df.isna().T.any().T]
nan_rows.head()

,body,score,subreddit


In [105]:
null_rows = df[df.isnull().any(axis=1)]
null_rows.head()

,body,score,subreddit


In [106]:
if np.isinf(df['score']).any():
  print("HELLO")

In [107]:
print(df.dtypes)

body         object
score         int64
subreddit    object
dtype: object


In [108]:
df.head()

,body,score,subreddit
0,that is nice of aa to do that . on a related n...,1,orangecounty
1,"the "" forgotten "" war is known by that moniker...",2,AskReddit
2,yet people build cobblestone generators .,1,MinecraftInventions
3,happy new year from england !,10,fffffffuuuuuuuuuuuu
4,[ there is one i might be able to get ] ( http...,1,cars


In [109]:
from sklearn.model_selection import train_test_split

#TODO: Complete this line using the train_test_split function and set the test size to .21
train_texts, val_texts, train_labels, val_labels = train_test_split(df['body'], df['subreddit'], test_size=0.21)


tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-base")
# Reset the index
train_texts, train_labels = train_texts.reset_index(drop=True), train_labels.reset_index(drop=True)
val_texts, val_labels = val_texts.reset_index(drop=True), val_labels.reset_index(drop=True)

In [110]:
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, return_tensors = "pt")
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, return_tensors = "pt")

In [111]:
id2label = dict(enumerate(set(df['subreddit'])))

In [112]:
id2label

{0: 'askseddit',
 1: 'LegalTeens',
 2: 'PS3',
 3: 'ponypapers',
 4: 'bayarea',
 5: 'happy',
 6: 'SF4',
 7: 'Idliketobeatree',
 8: 'comics',
 9: 'nosleep',
 10: 'Playdate',
 11: 'whatsthisbug',
 12: 'steamswap',
 13: 'GradSchool',
 14: 'Survival',
 15: 'Eugene',
 16: 'QuotesPorn',
 17: 'leangains',
 18: 'applehelp',
 19: 'ffxi',
 20: 'web_design',
 21: 'Judaism',
 22: 'spacedicks',
 23: 'radiohead',
 24: 'BabyBumps',
 25: 'Random_Acts_Of_Pizza',
 26: 'geek',
 27: 'progmetal',
 28: 'movies',
 29: 'WeAreTheMusicMakers',
 30: 'treecomics',
 31: 'Swapnote',
 32: 'HeroesofNewerth',
 33: 'circlejerk',
 34: 'thewalkingdead',
 35: 'AmISexy',
 36: 'Eve',
 37: 'AnythingGoesPics',
 38: 'centralpa',
 39: 'religion',
 40: 'Fallout',
 41: 'torrents',
 42: 'gif',
 43: 'leafs',
 44: 'stopdrinking',
 45: 'Libertarian',
 46: 'idolmaster',
 47: 'bf3pc',
 48: 'AlienBlue',
 49: 'TheSimpsons',
 50: 'FTH',
 51: 'sad',
 52: 'sabres',
 53: 'webdev',
 54: 'tumblr',
 55: 'REDDITORSINRECOVERY',
 56: 'SteamGameSwap

In [113]:
id2label #number:word

{0: 'askseddit',
 1: 'LegalTeens',
 2: 'PS3',
 3: 'ponypapers',
 4: 'bayarea',
 5: 'happy',
 6: 'SF4',
 7: 'Idliketobeatree',
 8: 'comics',
 9: 'nosleep',
 10: 'Playdate',
 11: 'whatsthisbug',
 12: 'steamswap',
 13: 'GradSchool',
 14: 'Survival',
 15: 'Eugene',
 16: 'QuotesPorn',
 17: 'leangains',
 18: 'applehelp',
 19: 'ffxi',
 20: 'web_design',
 21: 'Judaism',
 22: 'spacedicks',
 23: 'radiohead',
 24: 'BabyBumps',
 25: 'Random_Acts_Of_Pizza',
 26: 'geek',
 27: 'progmetal',
 28: 'movies',
 29: 'WeAreTheMusicMakers',
 30: 'treecomics',
 31: 'Swapnote',
 32: 'HeroesofNewerth',
 33: 'circlejerk',
 34: 'thewalkingdead',
 35: 'AmISexy',
 36: 'Eve',
 37: 'AnythingGoesPics',
 38: 'centralpa',
 39: 'religion',
 40: 'Fallout',
 41: 'torrents',
 42: 'gif',
 43: 'leafs',
 44: 'stopdrinking',
 45: 'Libertarian',
 46: 'idolmaster',
 47: 'bf3pc',
 48: 'AlienBlue',
 49: 'TheSimpsons',
 50: 'FTH',
 51: 'sad',
 52: 'sabres',
 53: 'webdev',
 54: 'tumblr',
 55: 'REDDITORSINRECOVERY',
 56: 'SteamGameSwap

In [114]:
id2label_reversed = {v: k for k, v in id2label.items()}

In [115]:
id2label_reversed #word:number

{'askseddit': 0,
 'LegalTeens': 1,
 'PS3': 2,
 'ponypapers': 3,
 'bayarea': 4,
 'happy': 5,
 'SF4': 6,
 'Idliketobeatree': 7,
 'comics': 8,
 'nosleep': 9,
 'Playdate': 10,
 'whatsthisbug': 11,
 'steamswap': 12,
 'GradSchool': 13,
 'Survival': 14,
 'Eugene': 15,
 'QuotesPorn': 16,
 'leangains': 17,
 'applehelp': 18,
 'ffxi': 19,
 'web_design': 20,
 'Judaism': 21,
 'spacedicks': 22,
 'radiohead': 23,
 'BabyBumps': 24,
 'Random_Acts_Of_Pizza': 25,
 'geek': 26,
 'progmetal': 27,
 'movies': 28,
 'WeAreTheMusicMakers': 29,
 'treecomics': 30,
 'Swapnote': 31,
 'HeroesofNewerth': 32,
 'circlejerk': 33,
 'thewalkingdead': 34,
 'AmISexy': 35,
 'Eve': 36,
 'AnythingGoesPics': 37,
 'centralpa': 38,
 'religion': 39,
 'Fallout': 40,
 'torrents': 41,
 'gif': 42,
 'leafs': 43,
 'stopdrinking': 44,
 'Libertarian': 45,
 'idolmaster': 46,
 'bf3pc': 47,
 'AlienBlue': 48,
 'TheSimpsons': 49,
 'FTH': 50,
 'sad': 51,
 'sabres': 52,
 'webdev': 53,
 'tumblr': 54,
 'REDDITORSINRECOVERY': 55,
 'SteamGameSwap': 5

In [116]:
import torch as pt



class SpamDataset(pt.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels.values

    def __getitem__(self, idx):
        # TODO: finish this dictionary comprehention to get the correct tensors passed into our transformer.
        item = {key: pt.tensor(val[idx]) for key, val in self.encodings.items()}
        if len(self.labels) > 0:  # Check if labels array has any elements
            labels = self.labels[idx]
            print(labels)
            labels_converted = id2label_reversed[labels]
            print(labels_converted)
            item["labels"] = pt.tensor(labels_converted)
        return item
    def __len__(self):
        print(len(self.labels))
        return len(self.labels)

def list_of_dicts_to_dict_of_lists(d):
  dic = d[0]
  keys = dic.keys()
  values = [dic.values() for dic in d]
  return {k: list(v) for k,v in zip(keys, zip(*values))}
# TODO: Now create the train and val datasets with the class we created above. HINT look at the parameters SpamDataset has and think about what it does



In [117]:
Train_Dataset = SpamDataset(train_encodings, train_labels)


In [118]:
Val_Dataset = SpamDataset(val_encodings, val_labels)

In [119]:
!pip install datasets transformers torch evaluate nltk rouge_score

In [120]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate


In [121]:
from sklearn.metrics import accuracy_score

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)
    # TODO: Look up on Hufggingface how to create a compute metrics function so we can return accuracy (using labels and predictions) and store the training information in a variable eval_pred


In [122]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total number of training epochs, let's try 1 for now
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)



trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = Train_Dataset,
    eval_dataset = Val_Dataset,
    compute_metrics = compute_metrics
    #TODO: figure out what 5 params we need here. There should be 5 and the last is how we compute the metrics later . . .
)

7900


In [123]:
# TODO: Train the model (HINT: this is 1 short line of code just calling the trainer)
trainer.train()

7900
7900
7900
7900
7900
7900
7900
7900
7900
7900
7900
7900
7900
pics
520
gameswap
247
wow
443
AskReddit
494
pics
520
funny
346
IndieGaming
855
pics
520
atheism
792
OneY
357
gaming
438
twincitiessocial
345
CrohnsDisease
167
SteamGameSwap
56
funny
346
rpg
756


<ipython-input-116-3b90d61ab440>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: pt.tensor(val[idx]) for key, val in self.encodings.items()}


ValueError: The model did not return a loss from the inputs, only the following keys: last_hidden_state,pooler_output. For reference, the inputs it received are input_ids,attention_mask.

In [ ]:
#TODO use the evaluate method on the trainer to get and print the results. Feel free to look at Huggingface Docs
results = trainer.evaluate()

In [ ]:
# TODO: Save the model
trainer.save_model('my_model')

In [ ]:
# TODO: Now to finish off I want you to load the model you trained and saved and write a fake spam email for it mthen have the model classify it
model = AutoModelForSequenceClassification.from_pretrained("my_model")
msg = "i want help to fix my car"
msg_encodings = tokenizer(msg, truncation=True, padding=True, return_tensors = "pt")

outputs = model(**msg_encodings)
print(f"outputs: {outputs}")
